# Decomposing Multiplicative Time Series in GCP BigQuery (and python too)

> Warning: UNDER CONSTRUCTION!!!

## Intro to blog post, what will be covered

## Time Series Stuff
### When time for code, ad link to skip setup

### Step 1. Create a project in GCP
- This must be globally unique across all of GCP. My project is called bq-timeseries-proj-51620.

### Step 2. Create a dataset in BigQuery
- From the Navigation menu in the upper left-hand corner, scroll down to BIG DATA and select BigQuery.
- On the left in the Navigation Pane, find your project and click on it.

![bq-proj](images/bq-projects.png)
- In the bottom pane on the right-hand side, click ![create_dataset](images/create_ds.png)
- Name your dataset, "timeseries". Leave all other defaults and select "Create dataset" at the bottom of the screen.
![name-dataset](images/name-dataset.png)
 - >Note: Note that Data location is important if you want to interact with other datasets and or projects. BigQuery requires that they be in the same location. This is outside the scope of this post, but may be important to note in future projcets.
- You should now see the timeseries dataset below your project in the left-hand pane.

![show-dataset](images/show-dataset.png)

### Step 3. Upload the AirPassengers csv to a BigQuery table.
- First, click on the dataset timeseries that you created in the above step and select ![create-table](images/create-table.png)
- Select Upload from the "Create table from:" dropdown, and browse to select the AirPassengers.csv (also located at link here). Make sure your project and dataset are correct. Name the table raw_airpassengers. Check Auto detect under Schema. Leave all other defualts and click Create table ![upload-table](images/upload-table.png)
- Now we are ready to start coding.


### Step 4. Clean the data
- Back in the left-hand pane, you should be able to click on your dataset and see the raw_airpassengers table below it. Click on the table and you will be able to see the table's Schema. ![raw-schema](images/raw-schema.png)
- If you click Preview, you can see the data and why the `Month` column has the `String` datatype. ![raw-data](images/raw-data.png)

- Trying to simply cast the Month column as a date will result in an error ![invalide-date](images/invalid-date.png)

- Because we are only concerned with the monthly data, we can concatenate `'-01'` to the end of the `Month` column and convert or cast this as a date. And because we don't want to have to do this step everytime we want to query the data, we will save our results to a table. I've also renamed the `Month` column to `mnth`. It is generally a best practice to avoid reserved words as column names. 

```sql
CREATE OR REPLACE TABLE
  `timeseries.airpassengers` AS
SELECT
  CAST(Month || '-01' AS DATE) AS mnth,
  Passengers
FROM
  `timeseries.raw_airpassengers`
```

In [24]:
#hide
import pandas as pd
bq = pd.read_csv('data/bq-timeseries.csv')
bq['recontructed'] = bq['recontructed'].astype(pd.Int64Dtype())
bq['Month'] = pd.to_datetime(bq['Month'])

### Step 5. Extract the Trend
- The trend in this context is a centered moving average. We can use the `OVER()` clause with the `AVG()` function in BigQuery to accomplish this. We believe the periodicity to be yearly/12 months. So we must center the average around this window. We will opt to use 6 months before and 5 months after the actual month. This includes the actual month making the period a full year. You can read more about the `OVER()` clause in [BigQuery here](https://cloud.google.com/bigquery/docs/reference/standard-sql/analytic-function-concepts).

```sql
  SELECT
    mnth,
    passengers,
    AVG(CAST(Passengers AS NUMERIC)) 
      OVER(
        ORDER BY mnth 
        ROWS BETWEEN 6 PRECEDING AND 5 FOLLOWING
      ) AS trend
FROM
  `timeseries.airpassengers`
```

In [7]:
#hide_input
pd.read_clipboard().head(10)

,mnth,passengers,trend
0,1949-01-01,112,124.500000
1,1949-02-01,118,127.857143
2,1949-03-01,132,130.375000
3,1949-04-01,129,131.000000
4,1949-05-01,121,129.800000
5,1949-06-01,135,127.454545
6,1949-07-01,148,126.666667
7,1949-08-01,148,126.916667
8,1949-09-01,136,127.583333
9,1949-10-01,119,128.333333


But you will notice that all months are returned in the result set even if they do not have data 6 months before or after the actual month. These values are not truly centered and must be ommitted. I'll choose to solve this using [common table expressions](https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax#with_clause) to make multiple passes at the data.

```sql
WITH initial AS (
  SELECT
    mnth,
    passengers,
    AVG(CAST(Passengers AS NUMERIC)) 
      OVER(
        ORDER BY mnth 
        ROWS BETWEEN 6 PRECEDING AND 5 FOLLOWING
      ) AS trend
FROM
  `timeseries.airpassengers`)
SELECT
a.mnth
,a.passengers
,a.trend
FROM initial a
WHERE EXISTS (
            SELECT 1 FROM initial x
            WHERE a.mnth = DATE_SUB(x.mnth, INTERVAL 6 MONTH)
            )
AND EXISTS (
          SELECT 1 FROM initial x
            WHERE a.mnth = DATE_ADD(x.mnth, INTERVAL 6 MONTH)
            )
```

In [8]:
#hide_input
pd.read_clipboard().head()

,mnth,passengers,trend
0,1949-07-01,148,126.666667
1,1949-08-01,148,126.916667
2,1949-09-01,136,127.583333
3,1949-10-01,119,128.333333
4,1949-11-01,104,128.833333


### Step 6. Detrend the data
- Removing the trend from the data will leave use with the seasonality. We believe our time series to be multiplicative, so we will divide the passengers column by the trend.

```sql
WITH initial AS (
  SELECT
    mnth,
    passengers,
    AVG(CAST(Passengers AS NUMERIC)) 
      OVER(
        ORDER BY mnth 
        ROWS BETWEEN 6 PRECEDING AND 5 FOLLOWING
      ) AS trend
FROM
  `timeseries.airpassengers`),
trend AS (
  SELECT
    a.mnth
    ,a.passengers
    ,a.trend
  FROM initial a
  WHERE EXISTS (
                SELECT 1 FROM initial x
                WHERE a.mnth = DATE_SUB(x.mnth, INTERVAL 6 MONTH)
                )
  AND EXISTS (
              SELECT 1 FROM initial x
                WHERE a.mnth = DATE_ADD(x.mnth, INTERVAL 6 MONTH)
                )
)             
SELECT
t.mnth
,t.passengers
,t.trend
,CAST(t.passengers AS NUMERIC) / t.trend AS detrend
FROM trend t
```

In [10]:
#hide_input
pd.read_clipboard().head(5)

,mnth,passengers,trend,detrend
0,1949-07-01,148,126.666667,1.168421
1,1949-08-01,148,126.916667,1.166120
2,1949-09-01,136,127.583333,1.065970
3,1949-10-01,119,128.333333,0.927273
4,1949-11-01,104,128.833333,0.807245


### Step 7. Extract the Random Noise
-  To extract the random noise, we need to remove the seasonality. Seasonality is the average of the detrended data grouped by month in our case. And because our SQL query is getting pretty long, I'll but cutting it up in the examples, but a complete sql script can be found POST LINK TO SQL HERE.

```sql
), detrend AS (              
  SELECT
    t.mnth
    ,t.passengers
    ,t.trend
    ,CAST(t.passengers AS NUMERIC) / t.trend AS detrend
  FROM trend t
)
SELECT
  d.mnth
  ,d.passengers
  ,d.trend
  ,d.detrend
  ,AVG(d.detrend) OVER (PARTITION BY EXTRACT(MONTH FROM d.mnth)) AS avg_seasonality
FROM detrend d
```

In [11]:
#hide_input
pd.read_clipboard().head(12)

,mnth,passengers,trend,detrend,avg_seasonality
0,1949-07-01,148,126.666667,1.168421,1.230005
1,1949-08-01,148,126.916667,1.166120,1.222717
2,1949-09-01,136,127.583333,1.065970,1.063440
3,1949-10-01,119,128.333333,0.927273,0.924510
4,1949-11-01,104,128.833333,0.807245,0.803798
5,1949-12-01,118,129.166667,0.913548,0.902477
6,1950-01-01,115,130.333333,0.882353,0.914743
7,1950-02-01,126,132.166667,0.953342,0.887863
8,1950-03-01,141,134.000000,1.052239,1.010943
9,1950-04-01,135,135.833333,0.993865,0.978781


- Given our equation for multiplicative time series is `time series = trend * seasonality * random noise`, we simply isoloation the variable `random noise` to get `random noise = time series / (seasonality * trend)`

```sql
), seasonality AS (
SELECT
  d.mnth
  ,d.passengers
  ,d.trend
  ,d.detrend
  ,AVG(d.detrend) OVER (PARTITION BY EXTRACT(MONTH FROM d.mnth)) AS avg_seasonality
FROM detrend d
)
SELECT
  s.mnth
  ,s.Passengers
  ,s.trend
  ,s.detrend
  ,s.avg_seasonality
  ,s.Passengers / (s.avg_seasonality * s.trend) AS random_noise
FROM seasonality s
```

In [12]:
#hide_input
pd.read_clipboard().head(10)

,mnth,Passengers,trend,detrend,avg_seasonality,random_noise
0,1949-07-01,148,126.666667,1.168421,1.230005,0.949932
1,1949-08-01,148,126.916667,1.166120,1.222717,0.953712
2,1949-09-01,136,127.583333,1.065970,1.063440,1.002379
3,1949-10-01,119,128.333333,0.927273,0.924510,1.002988
4,1949-11-01,104,128.833333,0.807245,0.803798,1.004287
5,1949-12-01,118,129.166667,0.913548,0.902477,1.012268
6,1950-01-01,115,130.333333,0.882353,0.914743,0.964592
7,1950-02-01,126,132.166667,0.953342,0.887863,1.073748
8,1950-03-01,141,134.000000,1.052239,1.010943,1.040849
9,1950-04-01,135,135.833333,0.993865,0.978781,1.015411


---
Now that we have all of the parts of our time series, we can actually reconstruct the original time series using each of its parts given our equation: `time series = trend * seasonality * random noise`. While we are at it, we will go back and get the portion of our data that we omitted.

```sql
), time_series AS (
  SELECT
    s.mnth
    ,s.Passengers
    ,s.trend
    ,s.detrend
    ,s.avg_seasonality
    ,s.Passengers / (s.avg_seasonality * s.trend) AS random_noise
  FROM seasonality s
)
SELECT
  a.mnth AS Month
  ,a.Passengers 
  ,t.trend
  ,t.detrend
  ,t.avg_seasonality
  ,t.random_noise
  ,CAST(ROUND(t.trend * t.avg_seasonality * t.random_noise,0) AS INT64) AS recontructed
FROM `timeseries.airpassengers` a
LEFT JOIN time_series t ON a.mnth= t.mnth
ORDER BY 1
```

In [25]:
#hide_input
bq.head(12)

,Month,Passengers,trend,detrend,avg_seasonality,random_noise,recontructed
0,1949-01-01,112,NaN,NaN,NaN,NaN,<NA>
1,1949-02-01,118,NaN,NaN,NaN,NaN,<NA>
2,1949-03-01,132,NaN,NaN,NaN,NaN,<NA>
3,1949-04-01,129,NaN,NaN,NaN,NaN,<NA>
4,1949-05-01,121,NaN,NaN,NaN,NaN,<NA>
5,1949-06-01,135,NaN,NaN,NaN,NaN,<NA>
6,1949-07-01,148,126.666667,1.168421,1.230005,0.949932,148
7,1949-08-01,148,126.916667,1.166120,1.222717,0.953712,148
8,1949-09-01,136,127.583333,1.065970,1.063440,1.002379,136
9,1949-10-01,119,128.333333,0.927273,0.924510,1.002988,119


As you can see, we were able to recreate the original time series values.